In [2]:
import pickle
new_df = pickle.load(open('new.pkl','rb'))
movies = pickle.load(open('movie_dict.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))
print(similarity[0])

[1.         0.08458258 0.05812382 ... 0.02478408 0.02739983 0.        ]


In [4]:
def recommend_10(movie):
    movie_list = []
    index = new_df[new_df['title'] == movie].index[0]   #index of searched movie in df
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])  #index and cos_sim tuple
    for i in distances[1:11]:
        movie = new_df.iloc[i[0]].title
        movie_list.append(movie)
    return movie_list

In [6]:
from flask import Flask,render_template,redirect,request,url_for

app = Flask(__name__)

@app.route('/')
def search():
    print("in /")
    return render_template('search.html')

@app.route('/submit', methods=['GET','POST'])
def submit():
    movie=''
    if request.method == 'POST':
        movie = request.form['search']
        movie_list = recommend_10(movie)
    return render_template('results.html',movies=movie_list)

@app.route('/details')
def details():
    movie = request.args.get('movie')
    return render_template('movie.html',movie = movie)


if __name__ == '__main__':
    print("in main")
    app.run()



in main
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [21/Feb/2024 19:26:58] "GET / HTTP/1.1" 200 -


in /


127.0.0.1 - - [21/Feb/2024 19:27:01] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2024 19:27:01] "GET /details?movie=Small%20Soldiers HTTP/1.1" 200 -


In [ ]:
p = recommend_10("Aliens")
print(p)